In [ ]:
import torch
# import intel_extension_for_pytorch as ipex

/home/girish/.local/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libjpeg.so.8: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    DistilBertForSequenceClassification,DistilBertTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, classification_report

2024-11-15 07:18:14.008380: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-15 07:18:14.014671: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 07:18:14.143936: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 07:18:14.143993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 07:18:14.163401: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [ ]:
# Load the cleaned training data
# train_df = pd.read_csv("train_cleaned_consolidated.csv")
train_df = pd.read_csv("new.csv")

In [4]:
train_df.shape

(13176, 2)

In [5]:
train_df.head()

,label,combined
0,Mostly False,Six out of 10 of the highest unemployment rate...
1,True,No Democratic campaign for (Fla.) governor has...
2,True,Says Ron Paul insisted FEMA should be shut dow...
3,Mostly False,"Says Hillary Clinton has ""been in office and i..."
4,False,"In 1916, the U.S. government predicted that by..."


In [6]:
train_df["label"].unique()

array(['Mostly False', 'True', 'False', 'Mostly True', 'Unverified/Mixed'],
      dtype=object)

In [7]:
train_df["label"] = train_df["label"].replace({"Mostly False": "False", "Mostly True": "True"})
train_df["label"].value_counts()

label
False               8383
True                4320
Unverified/Mixed     473
Name: count, dtype: int64

In [8]:
train_df = train_df[train_df["label"] != "Unverified/Mixed"]
train_df["label"].value_counts()


label
False    8383
True     4320
Name: count, dtype: int64

In [10]:
# Fit and transform the training labels and transform the test labels
train_df["label"] = train_df["label"].replace({"True": 1, "False": 0})

/tmp/ipykernel_18603/21520905.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df["label"] = train_df["label"].replace({"True": 1, "False": 0})


In [11]:
# Filter rows with 'False' label
false_df = train_df[train_df["label"] == 0]

# Randomly sample 4063 rows from the 'False' label
false_to_remove = false_df.sample(n=4063, random_state=42)  # You can change the random_state for different randomization

# Remove those rows from the original DataFrame
train_df = train_df.drop(false_to_remove.index)
train_df["label"].value_counts()

label
1    4320
0    4320
Name: count, dtype: int64

In [12]:
# Split the data into training and test sets (stratified)
train_data, test_data = train_test_split(train_df, test_size=0.2, stratify=train_df["label"])

In [13]:
# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [14]:
# Load the BERT tokenizer

model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [15]:
# Tokenize the text column
def tokenize_function(examples):
    return tokenizer(examples["combined"], padding="max_length", truncation=True)

In [16]:
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6912 [00:00<?, ? examples/s]

Map:   0%|          | 0/1728 [00:00<?, ? examples/s]

In [17]:
# Load the DistilBERT model for classification (2 labels in this case)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model.eval()
# model = model.to("xpu")
# model = ipex.optimize(model)
# model = model.train()

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    eval_strategy="epoch",  # evaluation strategy
    learning_rate=2e-5,  # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,  # batch size for evaluation
    num_train_epochs=1,  # number of epochs
    weight_decay=0.01,  # strength of weight decay
    logging_dir="./logs",  # directory for logs
    load_best_model_at_end=True,
    save_strategy="epoch",
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

In [29]:
# Set up the Trainer
trainer = Trainer(
    model=model,  # model to train
    args=training_args,  # training arguments
    train_dataset=train_dataset,  # training dataset
    eval_dataset=test_dataset,  # evaluation dataset (test set)
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [30]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.606736


TrainOutput(global_step=216, training_loss=0.4747846744678639, metrics={'train_runtime': 821.4059, 'train_samples_per_second': 8.415, 'train_steps_per_second': 0.263, 'total_flos': 915614659510272.0, 'train_loss': 0.4747846744678639, 'epoch': 1.0})

In [ ]:
# Make predictions on the test set
predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(axis=-1)

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(test_dataset["label"], preds)
print(f"Accuracy: {accuracy}")

Accuracy: 0.6336805555555556


In [ ]:
# Print a detailed classification report
report = classification_report(test_dataset["label"], preds)
print(f"Classification Report:\n{report}")  # Save the trained model
model.save_pretrained("bert_classifier")
tokenizer.save_pretrained("bert_classifier")

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.65      0.64       864
           1       0.64      0.62      0.63       864

    accuracy                           0.63      1728
   macro avg       0.63      0.63      0.63      1728
weighted avg       0.63      0.63      0.63      1728



('bert_classifier/tokenizer_config.json',
 'bert_classifier/special_tokens_map.json',
 'bert_classifier/vocab.txt',
 'bert_classifier/added_tokens.json')